In [1]:
!pip install  bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.0 MB/s eta 0:00:00


In [5]:
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed

# Load the base model with device_map set to 'auto'
model = AutoModelForCausalLM.from_pretrained(
    "SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora_merge",
    device_map='auto'
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("SHASWATSINGH3101/Qwen2-0.5B-Instruct_lora_merge")
tokenizer.pad_token = tokenizer.eos_token

def gen(model, p, maxlen=100, sample=True):
    toks = tokenizer(p, return_tensors="pt").to(model.device)  # Ensure tokens are on the same device as the model
    res = model.generate(**toks, max_new_tokens=maxlen, do_sample=sample,
                         num_return_sequences=1, temperature=0.1, num_beams=1, top_p=0.95)
    return tokenizer.batch_decode(res, skip_special_tokens=True)



In [10]:
%%time
seed = 42
set_seed(seed)

index = 1
prompt = "Write a letter informing someone of potential legal action due to a dispute or violation."
in_data = f"Instruct: {prompt}\n{prompt}\nOutput:\n"

# Generate response
peft_model_res = gen(model, in_data, 259)
peft_model_output = peft_model_res[0].split('Output:\n')[1]

# Extract the relevant parts of the output
prefix, success, result = peft_model_output.partition('#End')

# Print the results
dash_line = '-' * 100
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:
Write a letter informing someone of potential legal action due to a dispute or violation.
----------------------------------------------------------------------------------------------------
PEFT MODEL:
[Your Name]
[Your Address]
[City, State, ZIP Code]
[Email Address]
[Phone Number]

[Date]

[Recipient's Name]
[Recipient's Address]
[City, State, ZIP Code]


Subject: Informing Someone of Potential Legal Action Due to a Dispute or Violation

Dear [Recipient's Name],

I hope this letter finds you well. I am writing to inform you that there has been an issue involving my property, and it is important for me to take immediate action to address the situation.

As per our agreement, we have agreed upon the following steps:

1. Immediate communication with your attorney regarding the matter
2. Requested a meeting to discuss the situation further
3. Provided all relevant documenta

In [7]:
%%time
seed = 42
set_seed(seed)

index = 1
prompt = "Write a letter requesting a re-evaluation of your exam scores or grades."
in_data = f"Instruct: {prompt}\n{prompt}\nOutput:\n"

# Generate response
peft_model_res = gen(model, in_data, 259)
peft_model_output = peft_model_res[0].split('Output:\n')[1]

# Extract the relevant parts of the output
prefix, success, result = peft_model_output.partition('#End')

# Print the results
dash_line = '-' * 100
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

----------------------------------------------------------------------------------------------------
INPUT PROMPT:
Write a letter requesting a re-evaluation of your exam scores or grades.
----------------------------------------------------------------------------------------------------
PEFT MODEL:
[Your Name]
[Your Address]
[City, State, ZIP Code]
[Email Address]
[Phone Number]

[Date]

[Recipient's Name]
[Recipient's Address]
[City, State, ZIP Code]


Subject: Request for Re-Evaluation of Exam Scores/Grades

Dear [Recipient's Name],

I hope this message finds you well. I am writing to request that my exam scores and/or grades be reviewed by the appropriate authority. As you may know, I have been taking [Course Name] at [University Name], and I would greatly appreciate it if you could provide me with an updated evaluation.

My current scores are as follows:

- [Score 1]
- [Score 2]
- [Score 3]

Please let me know how we can proceed from here. Thank you in advance for your time and co